In [ ]:
import pip
pip.main(['install', 'pandas'])
pip.main(['install', 'matplotlib'])
pip.main(['install', 'seaborn'])
pip.main(['install', 'scikit-learn'])

# CHARGEMENT DES BIBLIOTHEQUES

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

Chargement du jeu de données REF_ARTICLE.

In [3]:
dataset_path = "C:\\Users\\belga\\OneDrive\\Bureau\\EMBA Big Data\\Module Datascience\\Projet Data Science S2 2024\\REF_ARTICLE.csv"
data4 = pd.read_csv(dataset_path, sep='|', low_memory=False)
print(data4.shape)
print(list(data4.columns))

(665946, 4)
['CODEARTICLE', 'CODEUNIVERS', 'CODEFAMILLE', 'CODESOUSFAMILLE']


La ligne de code ci-dessous nous permet de connaître le détail des colonnes ainsi que le type de données constituant notre base (numérique ou caractères).

In [4]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665946 entries, 0 to 665945
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   CODEARTICLE      665946 non-null  object
 1   CODEUNIVERS      665946 non-null  object
 2   CODEFAMILLE      665946 non-null  object
 3   CODESOUSFAMILLE  665946 non-null  object
dtypes: object(4)
memory usage: 20.3+ MB


Aperçu de notre jeu de données. 

In [5]:
data4

,CODEARTICLE,CODEUNIVERS,CODEFAMILLE,CODESOUSFAMILLE
0,COUPON,COUPON,COUPON,COUPON
1,1001,U2,130,130040
2,100125,U1,010,010020
3,100150,U1,010,010020
4,10018,U1,100,100900
...,...,...,...,...
665941,407717,U3,120,120070
665942,407719,U3,120,120070
665943,407720,U3,120,120070
665944,407721,U3,120,120070


La ligne de code ci-dessous nous permet de supprimer l'index 0 car cette ligne ne renvoi que la mention "COUPON", quelque soit l'information recherchée.

In [6]:
data4.drop(0, inplace=True)

Vérification du traitement.

In [7]:
data4

,CODEARTICLE,CODEUNIVERS,CODEFAMILLE,CODESOUSFAMILLE
1,1001,U2,130,130040
2,100125,U1,010,010020
3,100150,U1,010,010020
4,10018,U1,100,100900
5,10019,U1,100,100900
...,...,...,...,...
665941,407717,U3,120,120070
665942,407719,U3,120,120070
665943,407720,U3,120,120070
665944,407721,U3,120,120070


Cette ligne de code nous permet d'identifier le nombre d'occurence de chaque code article, pour identifier les éventuels regroupements à effectuer. Nous allons également effectuer cette opération sur les autres colonnes de notre jeu de données.

In [8]:
#modalités variables:
data4['CODEARTICLE'].value_counts(dropna = False)

CODEARTICLE
1001      1
278616    1
277497    1
277502    1
299706    1
         ..
145641    1
145642    1
145643    1
145644    1
407723    1
Name: count, Length: 665945, dtype: int64

In [9]:
data4['CODEUNIVERS'].value_counts(dropna = False)

CODEUNIVERS
U1    274652
U2    236139
U3     72725
U4     66413
U0     16016
Name: count, dtype: int64

In [10]:
data4['CODEFAMILLE'].value_counts(dropna = False)

CODEFAMILLE
090    157747
010     65324
130     63055
120     60886
180     46685
080     36253
100     33971
140     33907
020     31685
220     30893
050     22395
030     22165
070     14484
110     11839
160      8561
190      6449
040      5176
400      3884
170      3570
900      2196
200      1613
230      1092
060       979
210       853
990       283
Name: count, dtype: int64

Cette ligne de code nous permet d'identifier le nombre d'occurence de chaque code famille, pour identifier les éventuels regroupements à effectuer. 

In [11]:
# Importer la librairie pandas
import pandas as pd

# Créer un DataFrame de test
data = {'CODEFAMILLE': ['90', '090']}
df = pd.DataFrame(data)

# Additionner les valeurs '90' et '090' pour les regrouper en une seule modalité '90'
df['CODEFAMILLE'] = df['CODEFAMILLE'].replace({'090': '90'})

# Agréger les données en fonction de la colonne 'CODEFAMILLE' pour voir la somme des occurrences de chaque modalité
aggregated_data = df.groupby('CODEFAMILLE').size()

# Afficher le résultat
print(aggregated_data)

CODEFAMILLE
90    2
dtype: int64


Vérification du traitement.

In [12]:
data4['CODEFAMILLE'].value_counts(dropna = False)

CODEFAMILLE
090    157747
010     65324
130     63055
120     60886
180     46685
080     36253
100     33971
140     33907
020     31685
220     30893
050     22395
030     22165
070     14484
110     11839
160      8561
190      6449
040      5176
400      3884
170      3570
900      2196
200      1613
230      1092
060       979
210       853
990       283
Name: count, dtype: int64

In [13]:
data4['CODESOUSFAMILLE'].value_counts(dropna = False)

CODESOUSFAMILLE
180090    44779
140010    23698
120060    23371
090800    23326
090070    21254
          ...  
100110        1
110110        1
900990        1
200060        1
180999        1
Name: count, Length: 343, dtype: int64

La ligne de code ci-dessous nous permet d'identifier la présence ou non de valeur(s) en doublons.

In [14]:
print(data4.duplicated().sum())
data1 = data4.drop_duplicates()

0


La ligne de code ci-dessous nous permet d'identifier la présence ou non de valeur(s) manquante(s)

In [15]:
pd.DataFrame(index=data4.columns, columns=['%_missing_values'], data= (data4.isna().sum().values / len(data4)*100))

,%_missing_values
CODEARTICLE,0.0
CODEUNIVERS,0.0
CODEFAMILLE,0.0
CODESOUSFAMILLE,0.0


Le traitement des valeurs aberrantes ne concernant que les variables numériques, nous n'allons pas procéder à ce traitement car aucune de nos variables n'est concernées sur ce dataframe.

In [17]:
data4.drop(columns=['CODESOUSFAMILLE'], inplace=True)

Vérification du traitement

In [18]:
data4

,CODEARTICLE,CODEUNIVERS,CODEFAMILLE
1,1001,U2,130
2,100125,U1,010
3,100150,U1,010
4,10018,U1,100
5,10019,U1,100
...,...,...,...
665941,407717,U3,120
665942,407719,U3,120
665943,407720,U3,120
665944,407721,U3,120


La ligne de code ci-dessous nous permet d'identifier la liste des noms de colonnes de notre jeu de données. Cette liste nous sera utile pour cibler les colonnes dans nos futurs traitements.

In [19]:
# Importer la librairie pandas
import pandas as pd

# Définition des variables catégorielles
var_cat = data4.select_dtypes(include=['object']).columns.tolist()
print(var_cat)

['CODEARTICLE', 'CODEUNIVERS', 'CODEFAMILLE']


Le traitement suivant est utilisé pour encoder les étiquettes catégorielles en valeurs numériques. Cette conversion est nécessaire pour entraîner un modèle d'apprentissage automatique.

In [ ]:
#from sklearn.preprocessing import LabelEncoder

#label_encoder = LabelEncoder()

#for col in var_cat:
    #data4[col] = label_encoder.fit_transform(data4[col])
#print(data4.head())